In [17]:
import os

In [18]:
%pwd

'/workspaces/chicken-disease-classifier-cnn'

In [22]:
os.chdir("/workspaces/chicken-disease-classifier-cnn")

In [23]:
%pwd

'/workspaces/chicken-disease-classifier-cnn'

In [24]:
from dataclasses import dataclass
from pathlib import Path

""" Data Ingestion Configuration Structure"""

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir : Path 
  source_url : str
  local_data_file : Path
  unzip_dir : Path

In [25]:
#utilizing the folders
from cnnClassifier.constants import * #yaml files paths as Variables
from cnnClassifier.utils.common import read_yaml, create_directories #created functions  

In [30]:
"""To manage configuration"""

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) # variables created in the constant file
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root]) 
    

    """ To get the data ingestion config details """

    def get_data_ingestion_config(self): # Reading the config file
        config = self.config.data_ingestion #data_ingestion from the config file

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir, 
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config


In [31]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__ (self,config: DataIngestionConfig):
        self.config = config
    
    """ 
    checking for data available in the local folder 
    if not, download and gives the details else passes info
    """
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file): 
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extracts the zip file into the data diretory
        functions returns  None

        """

        un_zip_path = self.config.unzip_dir
        os.makedirs(un_zip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(un_zip_path)


In [35]:

""" Pipe Line"""

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-26 19:59:47,788: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-06-26 19:59:47,791: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-26 19:59:47,792: INFO: common: created directory at: artifacts]
[2023-06-26 19:59:47,793: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-26 19:59:49,361: INFO: 1019977797: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FD2E:7779:2DD9FB:359AEE:6499EE32
Accept-Ranges: bytes
Date: Mon, 26 Jun 2023 19:59:49 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1273-QPG
X-Cache: MIS